In [ ]:

import pandas as pd
from parsel import Selector
from time import sleep
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import tkinter as tk

# Set up the UI
root = tk.Tk()
root.title("LinkedIn Scraper")
root.geometry("400x200")

# Function to scrape LinkedIn
def scrape_linkedin():
    # Set up the webdriver
    opts = Options()
    driver = webdriver.Chrome()

    # function to ensure all key data fields have a value
    def validate_field(field):
        # if field is present pass
        if field:
            pass
        # if field is not present print text
        else:
            field = 'No results'
        return field

    # Log in to LinkedIn
    driver.get('https://www.linkedin.com')
    sleep(10)
    username = driver.find_element(By.ID,'session_key')
    username.send_keys(email_entry.get())
    sleep(0.5)
    password = driver.find_element(By.ID,'session_password')
    password.send_keys(password_entry.get())
    sleep(0.5)
    sign_in_button = driver.find_element(By.XPATH,'//*[@type="submit"]')
    sign_in_button.click()
    sleep(15)

    # Get user input for company and location
    Companies = company_entry.get()
    Location = location_entry.get()

    # Scrape LinkedIn
    Jobdata = []
    lnks = []
    driver.get(f'https://www.google.com/search?q=site%3Awww.linkedin.com%2Fin+{Companies}+%22{Location}%22')
    time.sleep(random.uniform(2.5, 4.9))

    for _ in range(1):
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="botstuff"]/div/div/div/a/h3/div/span/span')))
            driver.find_element(By.XPATH,'//*[@id="botstuff"]/div/div/div/a/h3/div/span/span').click()
            time.sleep(random.uniform(2.5, 4.9))
        except Exception as e:
            pass
        time.sleep(5)

        linkedin_urls = [my_elem.get_attribute("href") for my_elem in WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@class='yuRUbf']/div/span/a[@href]")))]
        lnks.append(linkedin_urls)

        for x in lnks:
            for i in x:
                driver.get(i)
                time.sleep(random.uniform(2.5, 4.9))
                sel = Selector(text=driver.page_source)
                name = sel.xpath('//*[starts-with(@class, "text-heading-xlarge inline t-24 v-align-middle break-words")]/text()').extract_first()
                if name:
                    name = name.strip()
                job_title = sel.xpath('//*[starts-with(@class, "text-body-medium break-words")]/text()').extract_first()
                if job_title:
                    job_title = job_title.strip()
                try:
                    company = driver.find_element(By.XPATH,'//ul[@class="pv-text-details__right-panel"]').text
                except:
                    company = 'None'
                if company:
                    company = company.strip()
                location = sel.xpath('//*[starts-with(@class, "text-body-small inline t-black--light break-words")]/text()').extract_first()
                if location:
                    location = location.strip()

                driver.get(i+"/overlay/contact-info/")
                time.sleep(random.uniform(2.5, 4.9))
                lst = []
                elems = driver.find_elements(By.XPATH,"//a[@href]")
                for elem in elems:
                    cntct = elem.get_attribute("href")
                    if cntct.endswith('?nis=true') and cntct.endswith('?nis=true&'):
                        pass
                    else:
                        lst.append(cntct)

                print('\n')
                print('Name: ' + name)
                print('Job Title: ' + job_title)
                print('Company: '+ company)
                print('Location: ' + location)
                print('URL: ' + i)
                lst3 = []
                for m in set(lst[:4]):
                    lst3.append(m)
                print("Contact Info" , lst3)
                print('\n')

                data = {
                    'Name': name,
                    'Job Title' : job_title,
                    'Company': company,
                    'Location' : location,
                    'URL': i,
                    'Contact_Info': lst3
                }
                Jobdata.append(data)

    df = pd.DataFrame(Jobdata)

    # Keep only the rows that have the word "mail" in them
    df = df[df.apply(lambda row: row.astype(str).str.contains('mail').any(), axis=1)]

    # Write the updated dataframe to a new Excel file
    df.to_excel('Jobdata_linkedin_Leads_filtered.xlsx')

# Create labels and entry fields for UI
email_label = tk.Label(root, text="Email:")
email_label.pack()
email_entry = tk.Entry(root)
email_entry.pack()

password_label = tk.Label(root, text="Password:")
password_label.pack()
password_entry = tk.Entry(root, show="*")
password_entry.pack()

company_label = tk.Label(root, text="Company:")
company_label.pack()
company_entry = tk.Entry(root)
company_entry.pack()

location_label = tk.Label(root, text="Location:")
location_label.pack()
location_entry = tk.Entry(root)
location_entry.pack()

# Create button to start scraping
scrape_button = tk.Button(root, text="Scrape LinkedIn", command=scrape_linkedin)
scrape_button.pack()

root.mainloop()
